# Title
[]()

In [6]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
from plotly.subplots import make_subplots
import requests
import json
import re

In [4]:
with open("credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['am_ig_user_id']
access_token = credentials['am_ig_access_token']

# `update_ig_account_insights`

## Version 1

In [7]:
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    try:
        df = loadpickle(filename, csv_path)
        previous_since = df.sort_values('timestamp')
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter >= since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = response
    if filename:
        filename = f'{filename}_account_insights'
        try:
            savepickle(df, filename, 'sav', csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename)

https://graph.facebook.com/v15.0/17841401256046961/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677657600.0&since=1675238400.0
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Number of days of data: 57
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/silvialiftsweights_account_insights.sav
Time completed: 2023-03-09 12:49:03.466033
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/silvialiftsweights_account_insights.sav
Time completed: 2023-03-09 12:49:03.468023


## version 2

In [75]:

from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        previous_since = datetime.strptime(previous_since, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
        print('previous until:', previous_until)

    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()) | (previous_until.date() < until.date()):
        print('Fetching new account insights')
    # if since_parameter:
    #     url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    # else:
    #     url_without_token += f'&since={datetime.timestamp(since)}'
    #     since_parameter = since

    # url = url_without_token+'&access_token='+access_token
    # print(url_without_token)
    
    # response_json_dict = dict()
    # df_list = []
    # earliest_end_time = None
    # page = 1
    # while (since_parameter >= since):
    #     response = requests.get(url)
    #     print(f'Requesting page {page}...')
    #     print('\tResponse status code: ',response.status_code)
    #     response_json_dict[page] = response.json()
    #     if response.status_code//100 != 2: # Stop the function if there is an error in the request
    #         print(response_json_dict[page]['error'])
    #         break
    #     try:
    #         df_list.append(
    #             pd.concat([
    #             json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
    #             json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
    #             ], axis=1)
    #         )
    #     except:
    #         print('No data in request response for page', page)
    #     earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
    #     since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

    #     try:
    #         next_endpoint = response_json_dict[page]['paging']['previous']
    #         if next_endpoint+access_token != url:
    #             url = next_endpoint
    #         else:
    #             print('end')
    #             break
    #     except: 
    #         break
    #     page +=1
    # try:
    #     df = pd.concat(df_list)
    #     df = df.reset_index(drop=True)
    #     print('Number of days of data:',len(df))
    # except:
    #     df = response
    # if filename:
    #     try:
    #         savepickle(df, filename+'_df', 'sav', csv_path)
    #         savepickle(response_json_dict,filename,'sav',json_path)
    #     except:
    #         print('Unable to save outputs')
    return df, response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since=since, until=until, filename=filename)
# response_json_dict

# data = update_ig_account_insights(
#     ig_user_id, access_token, since=since, until=until, filename=filename)
# data

Time completed: 2023-03-09 13:57:31.405137
Time completed: 2023-03-09 13:57:31.407139
previous until: 2023-03-01 08:00:00+00:00


In [84]:
len(response_json_dict)

2

In [86]:
dict(zip([key+len(response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())).keys()

dict_keys([3, 4])

In [89]:
{**response_json_dict, **dict(zip([key+len(response_json_dict) for key in response_json_dict.keys()], response_json_dict.values()))}.keys()

dict_keys([1, 2, 3, 4])

## Version 3

In [93]:
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter >= since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = response
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [104]:
# Make GET request only for dates that have not been saved in previous queries
from datetime import time, datetime, timedelta
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    SH 2023-03-09 12:37
    Get the daily impressions and reach a given Instagram account. Load results that were previously saved.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    if filename:
        filename = f'{filename}_account_insights'
    try:
        df = loadpickle(filename+'.sav', csv_path)
        response_json_dict = loadpickle(filename+'.sav', json_path)
        last_json_page = max(response_json_dict.keys())
        previous_since = response_json_dict[last_json_page]['data'][0]['values'][0]['end_time']
        previous_since = datetime.strptime(previous_since, "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        
        previous_until = response_json_dict[1]['data'][0]['values'][-1]['end_time']
        previous_until = datetime.strptime(previous_until, "%Y-%m-%dT%H:%M:%S%z")
    except:
        pass
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if (previous_since.date() > since.date()):
        print(f'Fetching new account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            since=since, until=previous_since)
        df = pd.concat([df, new_insights_df])
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        
    # if (previous_until.date() < until.date()):

    # if since_parameter:
    #     url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    # else:
    #     url_without_token += f'&since={datetime.timestamp(since)}'
    #     since_parameter = since

    # url = url_without_token+'&access_token='+access_token
    # print(url_without_token)
    
    # response_json_dict = dict()
    # df_list = []
    # earliest_end_time = None
    # page = 1
    # while (since_parameter >= since):
    #     response = requests.get(url)
    #     print(f'Requesting page {page}...')
    #     print('\tResponse status code: ',response.status_code)
    #     response_json_dict[page] = response.json()
    #     if response.status_code//100 != 2: # Stop the function if there is an error in the request
    #         print(response_json_dict[page]['error'])
    #         break
    #     try:
    #         df_list.append(
    #             pd.concat([
    #             json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
    #             json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
    #             ], axis=1)
    #         )
    #     except:
    #         print('No data in request response for page', page)
    #     earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
    #     since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

    #     try:
    #         next_endpoint = response_json_dict[page]['paging']['previous']
    #         if next_endpoint+access_token != url:
    #             url = next_endpoint
    #         else:
    #             print('end')
    #             break
    #     except: 
    #         break
    #     page +=1
    # try:
    #     df = pd.concat(df_list)
    #     df = df.reset_index(drop=True)
    #     print('Number of days of data:',len(df))
    # except:
    #     df = response
    # if filename:
    #     try:
    #         savepickle(df, filename+'_df', 'sav', csv_path)
    #         savepickle(response_json_dict,filename,'sav',json_path)
    #     except:
    #         print('Unable to save outputs')
    return df, response_json_dict, new_insights_response_json_dict

since = '2023-02-01'
until = '2023-03-01'
filename = 'silvialiftsweights'

data, response_json_dict, new_insights_response_json_dict = update_ig_account_insights(
    ig_user_id, access_token, since='2023-01-01', until=until, filename=filename)


Time completed: 2023-03-09 14:37:34.968390
Time completed: 2023-03-09 14:37:34.968390
Fetching new account insights from 2023-01-01 to 2023-01-04
https://graph.facebook.com/v15.0/17841401256046961/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1672819200.0&since=1672560000.0
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Number of days of data: 7


In [105]:
new_insights_response_json_dict

{3: {'data': [{'name': 'impressions',
    'period': 'day',
    'values': [{'value': 13185, 'end_time': '2023-01-01T08:00:00+0000'},
     {'value': 12254, 'end_time': '2023-01-02T08:00:00+0000'},
     {'value': 30510, 'end_time': '2023-01-03T08:00:00+0000'},
     {'value': 19138, 'end_time': '2023-01-04T08:00:00+0000'}],
    'title': 'Impressions',
    'description': "The total number of times that the business account's media objects have been viewed.",
    'id': '17841401256046961/insights/impressions/day'},
   {'name': 'reach',
    'period': 'day',
    'values': [{'value': 5845, 'end_time': '2023-01-01T08:00:00+0000'},
     {'value': 8573, 'end_time': '2023-01-02T08:00:00+0000'},
     {'value': 8682, 'end_time': '2023-01-03T08:00:00+0000'},
     {'value': 7431, 'end_time': '2023-01-04T08:00:00+0000'}],
    'title': 'Reach',
    'description': "The total number of times that the business account's media objects have been uniquely viewed.",
    'id': '17841401256046961/insights/reach/d

In [106]:
response_json_dict[4]

{'data': [{'name': 'impressions',
   'period': 'day',
   'values': [{'value': 13340, 'end_time': '2022-12-29T08:00:00+0000'},
    {'value': 12182, 'end_time': '2022-12-30T08:00:00+0000'},
    {'value': 18559, 'end_time': '2022-12-31T08:00:00+0000'}],
   'title': 'Impressions',
   'description': "The total number of times that the business account's media objects have been viewed.",
   'id': '17841401256046961/insights/impressions/day'},
  {'name': 'reach',
   'period': 'day',
   'values': [{'value': 5085, 'end_time': '2022-12-29T08:00:00+0000'},
    {'value': 5158, 'end_time': '2022-12-30T08:00:00+0000'},
    {'value': 5936, 'end_time': '2022-12-31T08:00:00+0000'}],
   'title': 'Reach',
   'description': "The total number of times that the business account's media objects have been uniquely viewed.",
   'id': '17841401256046961/insights/reach/day'}],
 'paging': {'previous': 'https://graph.facebook.com/v15.0/17841401256046961/insights?access_token=EAAKEofVXnvEBADnuuiqY6NI5hZAmAmF3YBdopZ

In [107]:
response_json_dict[1]

{'data': [{'name': 'impressions',
   'period': 'day',
   'values': [{'value': 12382, 'end_time': '2023-02-01T08:00:00+0000'},
    {'value': 11649, 'end_time': '2023-02-02T08:00:00+0000'},
    {'value': 13558, 'end_time': '2023-02-03T08:00:00+0000'},
    {'value': 8538, 'end_time': '2023-02-04T08:00:00+0000'},
    {'value': 12797, 'end_time': '2023-02-05T08:00:00+0000'},
    {'value': 9225, 'end_time': '2023-02-06T08:00:00+0000'},
    {'value': 14321, 'end_time': '2023-02-07T08:00:00+0000'},
    {'value': 6518, 'end_time': '2023-02-08T08:00:00+0000'},
    {'value': 8438, 'end_time': '2023-02-09T08:00:00+0000'},
    {'value': 7437, 'end_time': '2023-02-10T08:00:00+0000'},
    {'value': 11863, 'end_time': '2023-02-11T08:00:00+0000'},
    {'value': 7469, 'end_time': '2023-02-12T08:00:00+0000'},
    {'value': 7706, 'end_time': '2023-02-13T08:00:00+0000'},
    {'value': 7705, 'end_time': '2023-02-14T08:00:00+0000'},
    {'value': 10405, 'end_time': '2023-02-15T08:00:00+0000'},
    {'value': 

# *End of Page*